<a href="https://colab.research.google.com/github/FaraazArsath/Akaike-Assignment/blob/main/001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Structured Data Assignment - Problem 1**

**Importing packages**

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import datetime
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score
from xgboost import XGBClassifier
import joblib

**Reading Train Data**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Reading Train dataset
df_train = pd.read_parquet("/content/gdrive/MyDrive/Structured_Data_Assignment /train.parquet")
print(df_train.shape)
df_train.head()

(3220868, 3)


,Patient-Uid,Date,Incident
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1


**Understanding and Preparing the training Data**

In [ ]:
# Viewing data properties
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3220868 entries, 0 to 29080911
Data columns (total 3 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Patient-Uid  object        
 1   Date         datetime64[ns]
 2   Incident     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 98.3+ MB


In [ ]:
# Checking for missing value
missing_values = df_train.isnull().sum()
print(missing_values)

Patient-Uid    0
Date           0
Incident       0
dtype: int64


In [ ]:
# Finding duplicates
total_duplicates = df_train.duplicated().sum()
print(f'Total number of duplicates in the train dataset: {total_duplicates}')

Total number of duplicates in the train dataset: 35571


In [ ]:
# Drop duplicates in df_train
df_train.drop_duplicates(inplace=True)
df_train.duplicated().sum()

0

In [ ]:
# Identifying the Unique Patients in training data set

num_unique_patients = df_train['Patient-Uid'].nunique()
print(f'Total unique patients: {num_unique_patients}')

Total unique patients: 27033


In [ ]:
# Identifying patients who has taken 'Target Drug'

target_drug_patients = df_train[df_train['Incident'] == 'TARGET DRUG']
unique_target_drug_patients = target_drug_patients['Patient-Uid'].unique()
print(f'Total unique patients who took Target Drug: {len(unique_target_drug_patients)}')

Total unique patients who took Target Drug: 9374


In [ ]:
# Creating a positive set to identify patients who have taken the "Target Drug"

target_drug_patients = df_train[df_train['Incident'] == 'TARGET DRUG']
eligibility_dates = target_drug_patients.groupby('Patient-Uid')['Date'].min().reset_index()
target_drug_patients = target_drug_patients.merge(eligibility_dates, on='Patient-Uid', suffixes=('', '_eligibility'))

# Check if there are at least 30 days of medical history before eligibility
eligible_patients = target_drug_patients[(target_drug_patients['Date'] - target_drug_patients['Date_eligibility']) >= pd.Timedelta(days=30)]
positive_set = df_train[df_train['Patient-Uid'].isin(eligible_patients['Patient-Uid'].unique())]

# Adding a 'label' column with values as 1
positive_set['label'] = 1
positive_set.head()

,Patient-Uid,Date,Incident,label
8,a0e9c384-1c7c-11ec-81a0-16262ee38c7f,2018-02-22,SYMPTOM_TYPE_6,1
22,a0e9c3b3-1c7c-11ec-ae8e-16262ee38c7f,2018-02-21,SYMPTOM_TYPE_6,1
23,a0e9c3e3-1c7c-11ec-a8b9-16262ee38c7f,2017-05-11,SYMPTOM_TYPE_10,1
29,a0e9c414-1c7c-11ec-889a-16262ee38c7f,2019-11-22,PRIMARY_DIAGNOSIS,1
32,a0e9c443-1c7c-11ec-9eb0-16262ee38c7f,2020-01-28,PRIMARY_DIAGNOSIS,1


In [ ]:
# Checking for unique patients in positive set
unique_patient_ids_positive = positive_set['Patient-Uid'].nunique()
print(f'Number of unique patient IDs in positive set: {unique_patient_ids_positive}')

Number of unique patient IDs in positive set: 8047


In [ ]:
# Creating a neagtive set

# Getting unique patient IDs who have not taken the "Target Drug"
unique_negative_patients = df_train[~df_train['Patient-Uid'].isin(eligible_patients['Patient-Uid'].unique())]['Patient-Uid'].unique()
negative_set = df_train[df_train['Patient-Uid'].isin(unique_negative_patients)]

# Add a 'label' column with values as 0
negative_set['label'] = 0

negative_set.head()

,Patient-Uid,Date,Incident,label
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,2019-03-09,PRIMARY_DIAGNOSIS,0
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,2015-05-16,PRIMARY_DIAGNOSIS,0
3,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,2018-01-30,SYMPTOM_TYPE_0,0
4,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,2015-04-22,DRUG_TYPE_0,0
8,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,2016-06-18,DRUG_TYPE_1,0


In [ ]:
# Checking for unique patients in positive set
unique_patient_ids_positive = negative_set['Patient-Uid'].nunique()

print(f'Number of unique patient IDs in negative set: {unique_patient_ids_positive}')

Number of unique patient IDs in negative set: 18986


In [ ]:
# Creating a dataframe by concatenating positive and negative set

df= pd.concat([positive_set, negative_set], ignore_index=True)
df.reset_index(drop=True, inplace=True)
df

,Patient-Uid,Date,Incident,label
0,a0e9c384-1c7c-11ec-81a0-16262ee38c7f,2018-02-22,SYMPTOM_TYPE_6,1
1,a0e9c3b3-1c7c-11ec-ae8e-16262ee38c7f,2018-02-21,SYMPTOM_TYPE_6,1
2,a0e9c3e3-1c7c-11ec-a8b9-16262ee38c7f,2017-05-11,SYMPTOM_TYPE_10,1
3,a0e9c414-1c7c-11ec-889a-16262ee38c7f,2019-11-22,PRIMARY_DIAGNOSIS,1
4,a0e9c443-1c7c-11ec-9eb0-16262ee38c7f,2020-01-28,PRIMARY_DIAGNOSIS,1
...,...,...,...,...
3185292,a0ebddfb-1c7c-11ec-883b-16262ee38c7f,2017-01-22,DRUG_TYPE_3,0
3185293,a0ebba98-1c7c-11ec-b036-16262ee38c7f,2018-08-31,DRUG_TYPE_7,0
3185294,a0ebc6e3-1c7c-11ec-a900-16262ee38c7f,2019-03-05,DRUG_TYPE_0,0
3185295,a0ebca6c-1c7c-11ec-807c-16262ee38c7f,2017-08-29,DRUG_TYPE_7,0


**Feature Engineering - Train dataset**

In [ ]:
# Create columns from 'Incident' using one-hot encoding
df = pd.get_dummies(df, columns=['Incident'], prefix='', prefix_sep='')
df.columns

Index(['Patient-Uid', 'Date', 'label', 'DRUG_TYPE_0', 'DRUG_TYPE_1',
       'DRUG_TYPE_10', 'DRUG_TYPE_11', 'DRUG_TYPE_12', 'DRUG_TYPE_13',
       'DRUG_TYPE_14', 'DRUG_TYPE_15', 'DRUG_TYPE_16', 'DRUG_TYPE_17',
       'DRUG_TYPE_18', 'DRUG_TYPE_2', 'DRUG_TYPE_3', 'DRUG_TYPE_4',
       'DRUG_TYPE_5', 'DRUG_TYPE_6', 'DRUG_TYPE_7', 'DRUG_TYPE_8',
       'DRUG_TYPE_9', 'PRIMARY_DIAGNOSIS', 'SYMPTOM_TYPE_0', 'SYMPTOM_TYPE_1',
       'SYMPTOM_TYPE_10', 'SYMPTOM_TYPE_11', 'SYMPTOM_TYPE_12',
       'SYMPTOM_TYPE_13', 'SYMPTOM_TYPE_14', 'SYMPTOM_TYPE_15',
       'SYMPTOM_TYPE_16', 'SYMPTOM_TYPE_17', 'SYMPTOM_TYPE_18',
       'SYMPTOM_TYPE_19', 'SYMPTOM_TYPE_2', 'SYMPTOM_TYPE_20',
       'SYMPTOM_TYPE_21', 'SYMPTOM_TYPE_22', 'SYMPTOM_TYPE_23',
       'SYMPTOM_TYPE_24', 'SYMPTOM_TYPE_25', 'SYMPTOM_TYPE_26',
       'SYMPTOM_TYPE_27', 'SYMPTOM_TYPE_28', 'SYMPTOM_TYPE_29',
       'SYMPTOM_TYPE_3', 'SYMPTOM_TYPE_4', 'SYMPTOM_TYPE_5', 'SYMPTOM_TYPE_6',
       'SYMPTOM_TYPE_7', 'SYMPTOM_TYPE_8', 'SYMPT

In [ ]:
# Dropping 'Target Drug' column
df = df.drop('TARGET DRUG', axis=1)

In [ ]:
# Aggregating total occurrences of each incident type for every patient.

# Selecting the relevant columns for grouping
grouping_columns = ['Patient-Uid', 'label'] + [col for col in df.columns if col.startswith('DRUG_TYPE_')] + [col for col in df.columns if col.startswith('SYMPTOM_TYPE_')] + [col for col in df.columns if col.startswith('TEST_TYPE_')] + ['PRIMARY_DIAGNOSIS']

# Grouping by 'Patient-Uid' and 'Target', then sum the one-hot encoded columns
data = df[grouping_columns].groupby(['Patient-Uid', 'label']).sum().reset_index()

data.head()


,Patient-Uid,label,DRUG_TYPE_0,DRUG_TYPE_1,DRUG_TYPE_10,DRUG_TYPE_11,DRUG_TYPE_12,DRUG_TYPE_13,DRUG_TYPE_14,DRUG_TYPE_15,...,SYMPTOM_TYPE_7,SYMPTOM_TYPE_8,SYMPTOM_TYPE_9,TEST_TYPE_0,TEST_TYPE_1,TEST_TYPE_2,TEST_TYPE_3,TEST_TYPE_4,TEST_TYPE_5,PRIMARY_DIAGNOSIS
0,a0db1e73-1c7c-11ec-ae39-16262ee38c7f,0,29,0,0,1,0,0,0,0,...,1,0,0,10,2,0,0,0,0,13
1,a0dc93f2-1c7c-11ec-9cd2-16262ee38c7f,0,8,27,0,0,0,0,0,0,...,0,0,0,1,4,0,0,0,0,7
2,a0dc94c6-1c7c-11ec-a3a0-16262ee38c7f,0,6,7,0,10,0,0,0,0,...,0,0,0,3,2,0,0,0,0,18
3,a0dc950b-1c7c-11ec-b6ec-16262ee38c7f,0,15,42,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4,a0dc9543-1c7c-11ec-bb63-16262ee38c7f,0,2,45,0,24,0,0,0,0,...,5,6,0,9,27,1,0,0,0,29


In [ ]:
# Creating new columns by calculating the total count of incidents related to drug types, symptom types, and test types for each patient.

data['Total_DRUG_TYPE'] = data.iloc[:, 2:22].sum(axis=1)
data['Total_SYMPTOM_TYPE'] = data.iloc[:, 22:52].sum(axis=1)
data['Total_TEST_TYPE'] = data.iloc[:, 52:58].sum(axis=1)
data.shape

(27033, 61)

In [ ]:
# Exploring label variable
data['label'].value_counts()

0    18986
1     8047
Name: label, dtype: int64

We observed an imbalance in our label variable, with a distribution of 0 occurring 18,986 times and 1 occurring 8,047 times. To address this, we applied SMOTE (Synthetic Minority Over-sampling Technique) for oversampling.

**SMOTE - oversampling**

In [ ]:
X = data.drop(['label', 'Patient-Uid'], axis=1)
y = data['label']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
resampled_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='label')], axis=1)
resampled_data = resampled_data.sample(frac=1, random_state=42)
print("Before Sampling:")
print(data['label'].value_counts())
print("\nAfter Sampling:")
print(resampled_data['label'].value_counts())

Before Sampling:
0    18986
1     8047
Name: label, dtype: int64

After Sampling:
1    18986
0    18986
Name: label, dtype: int64


**XGBoost Model Training and Evaluation**

In [ ]:
# Training model on a resampled data
X = resampled_data.drop(['label'], axis=1)
y = resampled_data['label']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

# Initializing the XGBoost model
model = XGBClassifier(random_state=42)

# Tuning hyperparameters using GridSearch Cross validation
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Getting the best model from GridSearchCV
best_model = grid_search.best_estimator_

# Predict on the validation set
y_pred = best_model.predict(X_val)

# Calculate F1-score
f1 = f1_score(y_val, y_pred)

# Calculate Confusion Matrix
conf_matrix = confusion_matrix(y_val, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\n")
print(f"False Positives (FP): {conf_matrix[0, 1]}")
print(f"False Negatives (FN): {conf_matrix[1, 0]}")
print("\n")
print(f"F1-score: {f1}")

Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300}

Confusion Matrix:
[[3788  932]
 [ 738 4035]]


False Positives (FP): 932
False Negatives (FN): 738


F1-score: 0.8285420944558523


**Saving Trained XGBoost Model**

In [ ]:
# Saving the model using joblib.dump
model_filename = 'xgboost_model.pkl'
joblib.dump(best_model, model_filename)


['xgboost_model.pkl']

**Reading Test Data**

In [ ]:
# Reading Test dataset
df_test = pd.read_parquet('/content/gdrive/MyDrive/Structured_Data_Assignment /test.parquet')  # Load test data
df_test

,Patient-Uid,Date,Incident
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2016-12-08,SYMPTOM_TYPE_0
1,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-10-17,DRUG_TYPE_0
2,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-12-01,DRUG_TYPE_2
3,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-12-05,DRUG_TYPE_1
4,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-11-04,SYMPTOM_TYPE_0
...,...,...,...
1372854,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-05-11,DRUG_TYPE_13
1372856,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2018-08-22,DRUG_TYPE_2
1372857,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-02-04,DRUG_TYPE_2
1372858,a10272c9-1c7c-11ec-b3ce-16262ee38c7f,2017-09-25,DRUG_TYPE_8


**Understanding and Preparing the test Data**

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1065524 entries, 0 to 1372859
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Patient-Uid  1065524 non-null  object        
 1   Date         1065524 non-null  datetime64[ns]
 2   Incident     1065524 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 32.5+ MB


In [ ]:
# Checking for missing value
missing_values = df_test.isnull().sum()
print(missing_values)

Patient-Uid    0
Date           0
Incident       0
dtype: int64


In [ ]:
# Find duplicates in test data
total_duplicates = df_test.duplicated().sum()
print(f'Total number of duplicates in test dataset: {total_duplicates}')

Total number of duplicates in test dataset: 12100


In [ ]:
# Dropping duplicates
df_test.drop_duplicates(inplace=True)
df_test.duplicated().sum()

0

In [ ]:
# Identifying the Unique Patients in test data set
num_unique_patients = df_test['Patient-Uid'].nunique()
print(f'Total unique patients: {num_unique_patients}')

Total unique patients: 11482


**Feature Engineering - Test dataset**

In [ ]:
# Create columns from 'Incident' using one-hot encoding
df_test = pd.get_dummies(df_test, columns=['Incident'], prefix='', prefix_sep='')
df_test.head()

,Patient-Uid,Date,DRUG_TYPE_0,DRUG_TYPE_1,DRUG_TYPE_10,DRUG_TYPE_11,DRUG_TYPE_12,DRUG_TYPE_13,DRUG_TYPE_14,DRUG_TYPE_15,...,SYMPTOM_TYPE_6,SYMPTOM_TYPE_7,SYMPTOM_TYPE_8,SYMPTOM_TYPE_9,TEST_TYPE_0,TEST_TYPE_1,TEST_TYPE_2,TEST_TYPE_3,TEST_TYPE_4,TEST_TYPE_5
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2016-12-08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-10-17,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-12-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2018-12-05,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,2017-11-04,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Select the relevant columns for grouping
grouping_columns_test = ['Patient-Uid'] + [col for col in df_test.columns if col.startswith('DRUG_TYPE_')] + [col for col in df_test.columns if col.startswith('SYMPTOM_TYPE_')] + [col for col in df_test.columns if col.startswith('TEST_TYPE_')] + ['PRIMARY_DIAGNOSIS']
data_test = df_test[grouping_columns_test].groupby('Patient-Uid').sum().reset_index()
data_test.shape

(11482, 56)

In [ ]:
# Creating columns as per train dataset

data_test['Total_DRUG_TYPE'] = data_test.iloc[:, 2:22].sum(axis=1)
data_test['Total_SYMPTOM_TYPE'] = data_test.iloc[:, 22:52].sum(axis=1)
data_test['Total_TEST_TYPE'] = data_test.iloc[:, 52:58].sum(axis=1)

# Adding column 'DRUG_TYPE_18' with value 0 as this incident not recorded in test dataset
data_test['DRUG_TYPE_18'] = 0

# Positioning column 'DRUG_TYPE_18' after 'DRUG_TYPE_17' as per train dataset
position = data_test.columns.get_loc('DRUG_TYPE_17')
data_test.insert(position + 1, 'DRUG_TYPE_18', data_test.pop('DRUG_TYPE_18'))
data_test.shape

(11482, 60)

**Generating predictions for patients in test.parquet by loading our Predictive model**

In [ ]:
# Load the model
loaded_model = joblib.load(model_filename)

# Step 2: Use the Loaded Model to Make Predictions
X_test = data_test.drop(['Patient-Uid'], axis=1)  # Features
y_test_pred = loaded_model.predict(X_test)  # Predict

# Step 3: Create DataFrame with 'Patient-Uid' and 'label'
predictions_df = pd.DataFrame({'Patient-Uid': data_test['Patient-Uid'], 'label': y_test_pred})
predictions_df

,Patient-Uid,label
0,a0f9e8a9-1c7c-11ec-8d25-16262ee38c7f,0
1,a0f9e9f9-1c7c-11ec-b565-16262ee38c7f,1
2,a0f9ea43-1c7c-11ec-aa10-16262ee38c7f,1
3,a0f9ea7c-1c7c-11ec-af15-16262ee38c7f,0
4,a0f9eab1-1c7c-11ec-a732-16262ee38c7f,0
...,...,...
11477,a102720c-1c7c-11ec-bd9a-16262ee38c7f,1
11478,a102723c-1c7c-11ec-9f80-16262ee38c7f,1
11479,a102726b-1c7c-11ec-bfbf-16262ee38c7f,0
11480,a102729b-1c7c-11ec-86ba-16262ee38c7f,1


**Saving Predictions**

In [ ]:
# Saving file
predictions_df.to_csv('final_submission.csv', index=False)

**Evaluating F1 score of our Model**

In [ ]:
print(f"F1-score of Predictive model: {f1}")

F1-score of Predictive model: 0.8285420944558523
